<a href="https://colab.research.google.com/github/HollyIrvine/mgmt590-HollyIrvine-classResources/blob/main/Team_Assignment_1_Product_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connect Colab to BigQuery

In [ ]:
from google.colab import auth
auth.authenticate_user()
print("--- Authenticated to Google Cloud ---")

--- Authenticated to Google Cloud ---


In [ ]:
# @title Setup
from google.cloud import bigquery
from google.colab import data_table
import bigframes.pandas as bpd

project = 'mgmt599-hollyirvine-lab1' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()

In [ ]:
from google.cloud import bigquery
import pandas as pd # Often used to work with query results

# --- Configuration ---
PROJECT_ID = "mgmt599-hollyirvine-lab1"
DATASET_ID = "lab1_eda"
TABLE_ID = "superstore-customer_id2"
FULL_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

# Initialize the BigQuery client
client = bigquery.Client(project=PROJECT_ID)

print(f"--- Connected to BigQuery project: {PROJECT_ID} ---")

--- Connected to BigQuery project: mgmt599-hollyirvine-lab1 ---


## Category Performance

In [ ]:
# prompt: Write a comprehensive query to compare category performance using `mgmt599-hollyirvine-lab1.lab1_eda.superstore-customer_id2`.
# Requirements:
# - Group by category and calculate:
#   - Total revenue (sales)
#   - Total profit
#   - Average profit margin
#   - Total quantity sold
#   - Number of unique products
#   - Number of orders
#   - Average order value
# - Calculate profit-to-sales ratio for each category
# - Include growth metrics if possible (year-over-year comparison using order_date)
# - Order by total profit descending
# - Format monetary values appropriately

import pandas as pd
# SQL query for category performance analysis
query = f"""
WITH CategoryPerformance AS (
    SELECT
        category,
        SUM(sales) AS total_revenue,
        SUM(profit) AS total_profit,
        AVG(profit / sales) AS average_profit_margin,
        SUM(quantity) AS total_quantity_sold,
        COUNT(DISTINCT product_name) AS unique_products,
        COUNT(DISTINCT order_id) AS number_of_orders,
        SUM(sales) / COUNT(DISTINCT order_id) AS average_order_value,
        SUM(profit) / SUM(sales) AS profit_to_sales_ratio,
        EXTRACT(YEAR FROM order_date) AS order_year
    FROM
        `{FULL_TABLE_ID}`
    GROUP BY
        category,
        order_year
)

-- Compare year-over-year performance
SELECT
    cp_current.category,
    cp_current.order_year,
    cp_current.total_revenue,
    cp_current.total_profit,
    cp_current.average_profit_margin,
    cp_current.total_quantity_sold,
    cp_current.unique_products,
    cp_current.number_of_orders,
    cp_current.average_order_value,
    cp_current.profit_to_sales_ratio,
    LAG(cp_current.total_revenue, 1, 0) OVER (PARTITION BY cp_current.category ORDER BY cp_current.order_year) AS previous_year_revenue,
    (cp_current.total_revenue - LAG(cp_current.total_revenue, 1, 0) OVER (PARTITION BY cp_current.category ORDER BY cp_current.order_year)) / NULLIF(LAG(cp_current.total_revenue, 1, 0) OVER (PARTITION BY cp_current.category ORDER BY cp_current.order_year), 0) AS revenue_growth_rate,
    LAG(cp_current.total_profit, 1, 0) OVER (PARTITION BY cp_current.category ORDER BY cp_current.order_year) AS previous_year_profit,
     (cp_current.total_profit - LAG(cp_current.total_profit, 1, 0) OVER (PARTITION BY cp_current.category ORDER BY cp_current.order_year)) / NULLIF(LAG(cp_current.total_profit, 1, 0) OVER (PARTITION BY cp_current.category ORDER BY cp_current.order_year), 0) AS profit_growth_rate
FROM
    CategoryPerformance cp_current
ORDER BY
    cp_current.order_year DESC, -- Show latest year first
    cp_current.total_profit DESC -- Order by total profit descending within each year
"""

# Run the query
query_job = client.query(query)

# Convert results to DataFrame
results_df = query_job.to_dataframe()

# Display the results
print("--- Category Performance Analysis ---")
print(results_df)

# Optional: Format monetary columns for better readability
monetary_cols = ['total_revenue', 'total_profit', 'average_order_value', 'previous_year_revenue', 'previous_year_profit']
for col in monetary_cols:
    if col in results_df.columns:
        results_df[col] = results_df[col].apply(lambda x: f"${x:,.2f}" if pd.notnull(x) else None)

# Optional: Format percentages
percentage_cols = ['average_profit_margin', 'profit_to_sales_ratio', 'revenue_growth_rate', 'profit_growth_rate']
for col in percentage_cols:
     if col in results_df.columns:
        # Handle potential division by zero or zero previous year values gracefully
        results_df[col] = results_df[col].apply(lambda x: f"{x:.2%}" if pd.notnull(x) else None)


print("\n--- Formatted Category Performance Analysis ---")
results_df

--- Category Performance Analysis ---
           category  order_year  total_revenue  total_profit  \
0        Technology        2022    271730.8110    50684.2566   
1   Office Supplies        2022    246097.1750    39736.6217   
2         Furniture        2022    215387.2692     3018.3913   
3        Technology        2021    226364.1800    39773.9920   
4   Office Supplies        2021    183939.9820    35061.2292   
5         Furniture        2021    198901.4360     6959.9531   
6        Technology        2020    162780.8090    33503.8670   
7   Office Supplies        2020    137233.4630    25099.5338   
8         Furniture        2020    170518.2370     3015.2029   
9   Office Supplies        2019    151776.4120    22593.4161   
10       Technology        2019    175278.2330    21492.8325   
11        Furniture        2019    157192.8531     5457.7255   

    average_profit_margin  total_quantity_sold  unique_products  \
0                0.162107                 2363              30

,category,order_year,total_revenue,total_profit,average_profit_margin,total_quantity_sold,unique_products,number_of_orders,average_order_value,profit_to_sales_ratio,previous_year_revenue,revenue_growth_rate,previous_year_profit,profit_growth_rate
0,Technology,2022,"$271,730.81","$50,684.26",16.21%,2363,304,526,$516.60,18.65%,"$226,364.18",20.04%,"$39,773.99",27.43%
1,Office Supplies,2022,"$246,097.18","$39,736.62",12.81%,7676,888,1272,$193.47,16.15%,"$183,939.98",33.79%,"$35,061.23",13.33%
2,Furniture,2022,"$215,387.27","$3,018.39",3.86%,2437,318,564,$381.89,1.40%,"$198,901.44",8.29%,"$6,959.95",-56.63%
3,Technology,2021,"$226,364.18","$39,773.99",13.43%,1698,273,389,$581.91,17.57%,"$162,780.81",39.06%,"$33,503.87",18.71%
4,Office Supplies,2021,"$183,939.98","$35,061.23",16.09%,5946,805,946,$194.44,19.06%,"$137,233.46",34.03%,"$25,099.53",39.69%
5,Furniture,2021,"$198,901.44","$6,959.95",3.95%,2193,284,476,$417.86,3.50%,"$170,518.24",16.65%,"$3,015.20",130.83%
6,Technology,2020,"$162,780.81","$33,503.87",16.77%,1489,250,338,$481.60,20.58%,"$175,278.23",-7.13%,"$21,492.83",55.88%
7,Office Supplies,2020,"$137,233.46","$25,099.53",12.88%,4715,722,778,$176.39,18.29%,"$151,776.41",-9.58%,"$22,593.42",11.09%
8,Furniture,2020,"$170,518.24","$3,015.20",4.15%,1775,269,371,$459.62,1.77%,"$157,192.85",8.48%,"$5,457.73",-44.75%
9,Office Supplies,2019,"$151,776.41","$22,593.42",13.44%,4569,720,746,$203.45,14.89%,$0.00,None,$0.00,None


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=results_df)

https://docs.google.com/spreadsheets/d/141Mz4tZFIXIAzzyzOVt9_Z6Rz6LFwXuqO6KVkeAqV08/edit#gid=0


## Product Profitability

In [ ]:
# prompt: analyze product profitability using the table `mgmt599-hollyirvine-lab1.lab1_eda.superstore-customer_id2`.
# Requirements:
# - Calculate total profit, total sales, average profit margin, and total quantity sold for each product
# - Calculate profit per unit (total profit / total quantity)
# - Rank products by total profit (highest to lowest)
# - Include only products with at least 10 orders to ensure statistical significance
# - Show top 20 most profitable products
# - Include columns: product_name, total_profit, total_sales, avg_profit_margin, total_quantity, profit_per_unit, order_count

import pandas as pd
# SQL query for product profitability analysis
product_query = f"""
SELECT
    product_name,
    SUM(profit) AS total_profit,
    SUM(sales) AS total_sales,
    AVG(profit / sales) AS avg_profit_margin,
    SUM(quantity) AS total_quantity,
    COUNT(DISTINCT order_id) AS order_count
FROM
    `{FULL_TABLE_ID}`
GROUP BY
    product_name
HAVING
    COUNT(DISTINCT order_id) >= 10 -- Filter for products with at least 10 orders
"""

# Run the product profitability query
product_query_job = client.query(product_query)

# Convert product results to DataFrame
product_results_df = product_query_job.to_dataframe()

# Calculate profit per unit
product_results_df['profit_per_unit'] = product_results_df['total_profit'] / product_results_df['total_quantity']

# Rank products by total profit (highest to lowest)
product_results_df = product_results_df.sort_values(by='total_profit', ascending=False)

# Select the top 20 most profitable products
top_20_profitable_products = product_results_df.head(20).copy()

# Display the results
print("\n--- Top 20 Most Profitable Products (with at least 10 orders) ---")
# Optional: Format monetary columns for better readability
monetary_cols_product = ['total_profit', 'total_sales', 'profit_per_unit']
for col in monetary_cols_product:
    if col in top_20_profitable_products.columns:
        top_20_profitable_products[col] = top_20_profitable_products[col].apply(lambda x: f"${x:,.2f}" if pd.notnull(x) else None)

# Optional: Format percentages
percentage_cols_product = ['avg_profit_margin']
for col in percentage_cols_product:
     if col in top_20_profitable_products.columns:
        top_20_profitable_products[col] = top_20_profitable_products[col].apply(lambda x: f"{x:.2%}" if pd.notnull(x) else None)


top_20_profitable_products


--- Top 20 Most Profitable Products (with at least 10 orders) ---


,product_name,total_profit,total_sales,avg_profit_margin,total_quantity,order_count,profit_per_unit
61,Fellowes PB500 Electric Punch Plastic Comb Bin...,"$7,753.04","$27,453.38",5.00%,31,10,$250.10
91,Plantronics CS510 - Over-the-Head monaural Wir...,"$3,085.03","$10,822.36",26.30%,35,10,$88.14
49,GBC DocuBind TL300 Electric Binding System,"$2,233.51","$19,823.48",-8.41%,37,11,$60.37
20,"Hot File 7-Pocket, Floor Stand","$2,016.71","$8,138.23",23.27%,49,13,$41.16
9,Adjustable Depth Letter/Legal Cart,"$1,932.55","$7,730.20",23.08%,45,11,$42.95
62,"Hon Deluxe Fabric Upholstered Stacking Chairs,...","$1,927.44","$10,637.53",16.83%,51,12,$37.79
6,Tennsco 6- and 18-Compartment Lockers,"$1,575.11","$10,925.00",12.41%,43,11,$36.63
122,Standard Rollaway File with Lock,"$1,261.33","$6,594.95",20.45%,40,10,$31.53
96,SAFCO Arco Folding Chair,"$1,179.37","$11,572.78",10.45%,53,13,$22.25
7,Easy-staple paper,"$1,096.03","$2,504.19",43.16%,150,46,$7.31


## Discount Performance

In [ ]:
# prompt: Write code to analyze the relationship between discounts and product performance using `mgmt599-hollyirvine-lab1.lab1_eda.superstore-customer_id2`.
# Requirements:
# - Create discount bands: 0%, 0-10%, 10-20%, 20-30%, 30%+
# - For each discount band, calculate:
#   - Average profit margin
#   - Total profit
#   - Total sales volume
#   - Average quantity per order
#   - Number of orders
# - Calculate the impact of discount on profit per unit
# - Use CASE statements to categorize discount levels
# - Show which discount ranges are most profitable
# - Include statistical measures like median profit margin per discount band

import pandas as pd
# SQL query to analyze the relationship between discounts and product performance
discount_query = f"""
SELECT
    -- Categorize discount levels using CASE statements
    CASE
        WHEN discount = 0 THEN '0%'
        WHEN discount > 0 AND discount <= 0.1 THEN '0-10%'
        WHEN discount > 0.1 AND discount <= 0.2 THEN '10-20%'
        WHEN discount > 0.2 AND discount <= 0.3 THEN '20-30%'
        WHEN discount > 0.3 THEN '30%+'
        ELSE 'Other' -- Should not happen with the given data, but good for robustness
    END AS discount_band,
    -- Calculate performance metrics for each discount band
    AVG(profit / sales) AS average_profit_margin,
    SUM(profit) AS total_profit,
    SUM(sales) AS total_sales_volume,
    AVG(quantity) AS average_quantity_per_order,
    COUNT(DISTINCT order_id) AS number_of_orders,
    AVG(profit / quantity) AS average_profit_per_unit, -- Impact of discount on profit per unit
    -- Include statistical measures like median profit margin per discount band
    APPROX_QUANTILES(profit / sales, 2)[OFFSET(1)] AS median_profit_margin
FROM
    `{FULL_TABLE_ID}`
GROUP BY
    discount_band
ORDER BY
    -- Order by total profit descending to show most profitable bands first
    total_profit DESC
"""

# Run the discount analysis query
discount_query_job = client.query(discount_query)

# Convert discount analysis results to DataFrame
discount_results_df = discount_query_job.to_dataframe()

# Display the results
print("\n--- Discount Level Performance Analysis ---")

# Optional: Format monetary columns for better readability
monetary_cols_discount = ['total_profit', 'total_sales_volume', 'average_profit_per_unit']
for col in discount_results_df.columns:
    if col in monetary_cols_discount:
        # Convert to numeric first, then format
        discount_results_df[col] = pd.to_numeric(discount_results_df[col], errors='coerce').apply(
            lambda x: f"-${abs(x):,.2f}" if pd.notnull(x) and x < 0
            else f"${x:,.2f}" if pd.notnull(x) else None
        )

# Optional: Format percentages and ratios
percentage_cols_discount = ['average_profit_margin', 'median_profit_margin']
for col in discount_results_df.columns:
    if col in percentage_cols_discount:
        discount_results_df[col] = discount_results_df[col].apply(lambda x: f"{x:.2%}" if pd.notnull(x) else None)

# Format average quantities to integers
discount_results_df['average_quantity_per_order'] = discount_results_df['average_quantity_per_order'].round().astype(int)

discount_results_df


--- Discount Level Performance Analysis ---


,discount_band,average_profit_margin,total_profit,total_sales_volume,average_quantity_per_order,number_of_orders,average_profit_per_unit,median_profit_margin
0,0%,34.02%,"$320,987.60","$1,087,908.47",4,2644,$17.58,33.00%
1,10-20%,17.48%,"$91,756.30","$792,152.89",4,2436,$6.80,16.25%
2,0-10%,15.58%,"$9,029.18","$54,369.35",4,89,$24.82,16.67%
3,20-30%,-11.55%,"-$10,369.28","$103,226.65",4,211,-$12.65,-8.57%
4,30%+,-91.47%,"-$125,006.78","$259,543.49",4,888,-$26.64,-73.33%


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=discount_results_df)

https://docs.google.com/spreadsheets/d/1opA7XhoUCv9zeXdf1LHhFOUphdAoqPzKhVqHddmt1zo/edit#gid=0


## Strategic Product Portfolio

In [ ]:
# prompt: 4.	Create a strategic product portfolio matrix using `mgmt599-hollyirvine-lab1.lab1_eda.superstore-customer_id2`.
# Requirements:
# - Calculate for each product:
# -- Total profit (profitability measure)
# -- Total sales volume (market size measure)
# -- Profit growth rate (if comparing time periods)
# -- Market share within category
# - Create a 2x2 matrix classification:
# -- High Profit + High Volume = "Stars"
# -- High Profit + Low Volume = "Cash Cows"
# -- Low Profit + High Volume = "Question Marks"
# -- Low Profit + Low Volume = "Dogs"
# -- Use CASE statements to assign each product to a quadrant
# - Calculate percentiles for profit and volume to determine high/low thresholds
# - Show count of products in each quadrant and their total contribution

import pandas as pd
# SQL query for product portfolio matrix
portfolio_query = f"""
WITH ProductMetrics AS (
    SELECT
        product_name,
        SUM(profit) AS total_profit,
        SUM(sales) AS total_sales,
        -- You can add growth rate calculations here if needed,
        -- potentially by joining or using window functions on year data.
        -- For simplicity, we'll focus on total profit and total sales volume.
        -- To calculate market share within category, you would need to join with
        -- a table or CTE that has total sales per category.
        -- Example (requires joining with a table that has category info):
        -- SUM(sales) / SUM(sales) OVER (PARTITION BY category) AS market_share_category
        'Placeholder Category' as category -- Replace with actual category if available
    FROM
        `{FULL_TABLE_ID}`
    GROUP BY
        product_name
),
ProfitPercentiles AS (
    SELECT
        product_name,
        total_profit,
        total_sales,
        category,
        NTILE(4) OVER (ORDER BY total_profit) AS profit_quartile
    FROM
        ProductMetrics
),
SalesPercentiles AS (
    SELECT
        product_name,
        total_profit,
        total_sales,
        category,
        profit_quartile,
        NTILE(4) OVER (ORDER BY total_sales) AS sales_quartile
    FROM
        ProfitPercentiles
)
SELECT
    product_name,
    total_profit,
    total_sales,
    category,
    profit_quartile,
    sales_quartile,
    CASE
        WHEN profit_quartile >= 3 AND sales_quartile >= 3 THEN 'Stars' -- High Profit, High Volume (Using >= 3 for top 50%)
        WHEN profit_quartile >= 3 AND sales_quartile < 3 THEN 'Cash Cows' -- High Profit, Low Volume
        WHEN profit_quartile < 3 AND sales_quartile >= 3 THEN 'Question Marks' -- Low Profit, High Volume
        WHEN profit_quartile < 3 AND sales_quartile < 3 THEN 'Dogs' -- Low Profit, Low Volume
        ELSE 'Other'
    END AS portfolio_quadrant
FROM
    SalesPercentiles
"""

# Run the portfolio analysis query
portfolio_query_job = client.query(portfolio_query)

# Convert portfolio analysis results to DataFrame
portfolio_results_df = portfolio_query_job.to_dataframe()

# Display the results
print("\n--- Strategic Product Portfolio Analysis ---")
portfolio_results_df

# Calculate counts and total contribution per quadrant
quadrant_summary = portfolio_results_df.groupby('portfolio_quadrant').agg(
    product_count=('product_name', 'count'),
    total_profit_contribution=('total_profit', 'sum'),
    total_sales_contribution=('total_sales', 'sum')
).reset_index()

# Display the quadrant summary
print("\n--- Product Portfolio Quadrant Summary ---")
quadrant_summary

# Optional: Format summary monetary columns for better readability
monetary_cols_summary = ['total_profit_contribution', 'total_sales_contribution']
for col in quadrant_summary.columns:
    if col in monetary_cols_summary:
        quadrant_summary[col] = quadrant_summary[col].apply(lambda x: f"${x:,.2f}" if pd.notnull(x) else None)

quadrant_summary



--- Strategic Product Portfolio Analysis ---

--- Product Portfolio Quadrant Summary ---


,portfolio_quadrant,product_count,total_profit_contribution,total_sales_contribution
0,Cash Cows,252,"$17,746.97","$49,168.97"
1,Dogs,673,"$8,638.53","$60,941.12"
2,Question Marks,252,"$-72,632.44","$613,464.49"
3,Stars,672,"$332,643.95","$1,573,626.28"


## Profitability Drivers

In [ ]:
# prompt: 5.	Create a query to identify key drivers of product profitability using `mgmt599-hollyirvine-lab1.lab1_eda.superstore-customer_id2`.
# Requirements:
# - Group by product_name and calculate:
# -- Average discount rate
# -- Average profit margin
# -- Total profit
# -- Average order value (sales/quantity)
# - Calculate correlation indicators between discount and profitability
# - Show products with highest profit margins vs. lowest discount rates
# - Include a profitability_score calculation: (total_profit * avg_profit_margin) / (1 + avg_discount)
# - Order by profitability_score descending - Limit to top 25 products

def analyze_product_profitability(client, table_id):
    """Analyze product profitability drivers with categories."""

    # Query including category
    query = f"""
    SELECT
        product_name,
        AVG(discount) AS avg_discount_rate,
        AVG(profit / sales) AS avg_profit_margin,
        SUM(profit) AS total_profit,
        SUM(sales) AS total_sales,
        SUM(sales) / COUNT(DISTINCT order_id) AS avg_order_value_per_order,
        (SUM(profit) * AVG(profit / sales)) / (1 + AVG(discount)) AS profitability_score
    FROM
        `{table_id}`
    GROUP BY
        product_name
    ORDER BY
        profitability_score DESC
    LIMIT 25
    """

    # Execute and format
    df = client.query(query).to_dataframe()

    # Create formatted copy
    df_display = df.copy()

    # Apply formatting
    format_rules = {
        'total_profit': '${:,.2f}',
        'avg_order_value_per_order': '${:,.2f}',
        'profitability_score': '${:,.2f}',
        'avg_discount_rate': '{:.2%}',
        'avg_profit_margin': '{:.2%}'
    }

    for col, fmt in format_rules.items():
        if col in df_display.columns:
            df_display[col] = df[col].apply(lambda x: fmt.format(x) if pd.notnull(x) else None)

    # Calculate correlations
    numeric_cols = ['avg_discount_rate', 'avg_profit_margin', 'total_profit',
                    'avg_order_value_per_order', 'profitability_score']
    correlations = df[numeric_cols].corr()

    # High margin products
    high_margin_products = df.nlargest(10, 'avg_profit_margin')

    return df_display, correlations, high_margin_products

# Usage
products_df, corr_matrix, high_margin = analyze_product_profitability(client, FULL_TABLE_ID)

print("\n--- Product Profitability Analysis ---")
print(products_df)
print("\n--- Correlations ---")
print(corr_matrix)


--- Product Profitability Analysis ---
                                         product_name avg_discount_rate  \
0               Canon imageCLASS 2200 Advanced Copier            12.00%   
1           Cubify CubeX 3D Printer Double Head Print            53.33%   
2                Hewlett Packard LaserJet 3310 Copier            20.00%   
3                   Ativa V4110MDD Micro-Cut Shredder             0.00%   
4                   Zebra ZM400 Thermal Label Printer             0.00%   
5                  Canon PC1060 Personal Laser Copier            15.00%   
6   Plantronics Savi W720 Multi-Device Wireless He...             5.71%   
7           Cubify CubeX 3D Printer Triple Head Print            50.00%   
8           Lexmark MX611dhe Monochrome Laser Printer            40.00%   
9   Canon imageCLASS MF7460 Monochrome Digital Las...             0.00%   
10   3D Systems Cube Printer, 2nd Generation, Magenta             0.00%   
11  Hewlett Packard 610 Color Digital Copier / Pri...       

In [ ]:
# prompt: 14.	Create a strategic product portfolio matrix, divided by region, using `mgmt599-hollyirvine-lab1.lab1_eda.superstore-customer_id2`.
# Requirements:
# - Calculate for each product:
# -- Total profit (profitability measure)
# -- Total sales volume (market size measure)
# -- Profit growth rate (if comparing time periods)
# -- Market share within category
# For each region:
# - Create a 2x2 matrix classification:
# -- High Profit + High Volume = "Stars"
# -- High Profit + Low Volume = "Cash Cows"
# -- Low Profit + High Volume = "Question Marks"
# -- Low Profit + Low Volume = "Dogs"
# -- Use CASE statements to assign each product to a quadrant
# - Calculate percentiles for profit and volume to determine high/low thresholds
# - Show count of products in each quadrant and their total contribution

import pandas as pd
# SQL query for strategic product portfolio matrix by region
portfolio_query_region = f"""
WITH RegionalProductMetrics AS (
    SELECT
        region,
        product_name,
        SUM(profit) AS total_profit,
        SUM(sales) AS total_sales,
        -- Add product category here if available in the original table
        -- category -- Example: category field from the original table
    FROM
        `{FULL_TABLE_ID}`
    GROUP BY
        region,
        product_name
),
RegionalProfitPercentiles AS (
    SELECT
        region,
        product_name,
        total_profit,
        total_sales,
        -- category, -- Example: category field
        NTILE(4) OVER (PARTITION BY region ORDER BY total_profit) AS profit_quartile_region
    FROM
        RegionalProductMetrics
),
RegionalSalesPercentiles AS (
    SELECT
        region,
        product_name,
        total_profit,
        total_sales,
        -- category, -- Example: category field
        profit_quartile_region,
        NTILE(4) OVER (PARTITION BY region ORDER BY total_sales) AS sales_quartile_region
    FROM
        RegionalProfitPercentiles
)
SELECT
    region,
    product_name,
    total_profit,
    total_sales,
    -- category, -- Example: category field
    profit_quartile_region,
    sales_quartile_region,
    CASE
        WHEN profit_quartile_region >= 3 AND sales_quartile_region >= 3 THEN 'Stars' -- High Profit, High Volume (Using >= 3 for top 50%)
        WHEN profit_quartile_region >= 3 AND sales_quartile_region < 3 THEN 'Cash Cows' -- High Profit, Low Volume
        WHEN profit_quartile_region < 3 AND sales_quartile_region >= 3 THEN 'Question Marks' -- Low Profit, High Volume
        WHEN profit_quartile_region < 3 AND sales_quartile_region < 3 THEN 'Dogs' -- Low Profit, Low Volume
        ELSE 'Other'
    END AS portfolio_quadrant_region
FROM
    RegionalSalesPercentiles
ORDER BY
    region,
    portfolio_quadrant_region DESC
"""

# Run the regional portfolio analysis query
portfolio_query_region_job = client.query(portfolio_query_region)

# Convert regional portfolio analysis results to DataFrame
portfolio_results_region_df = portfolio_query_region_job.to_dataframe()

# Display the results
print("\n--- Strategic Product Portfolio Analysis by Region ---")
portfolio_results_region_df

# Calculate counts and total contribution per quadrant and region
quadrant_region_summary = portfolio_results_region_df.groupby(['region', 'portfolio_quadrant_region']).agg(
    product_count=('product_name', 'count'),
    total_profit_contribution=('total_profit', 'sum'),
    total_sales_contribution=('total_sales', 'sum')
).reset_index()

# Display the regional quadrant summary
print("\n--- Product Portfolio Quadrant Summary by Region ---")

# Optional: Format summary monetary columns for better readability
monetary_cols_summary_region = ['total_profit_contribution', 'total_sales_contribution']
for col in quadrant_region_summary.columns:
    if col in monetary_cols_summary_region:
        quadrant_region_summary[col] = quadrant_region_summary[col].apply(
            lambda x: f"${x:,.2f}" if pd.notnull(x) else None
        )

quadrant_region_summary



--- Strategic Product Portfolio Analysis by Region ---

--- Product Portfolio Quadrant Summary by Region ---


,region,portfolio_quadrant_region,product_count,total_profit_contribution,total_sales_contribution
0,Central,Cash Cows,214,"$3,493.80","$9,953.43"
1,Central,Dogs,434,"$-4,091.29","$10,209.58"
2,Central,Question Marks,214,"$-33,881.28","$152,316.14"
3,Central,Stars,432,"$74,185.14","$328,760.74"
4,East,Cash Cows,197,"$4,951.07","$13,121.49"
5,East,Dogs,511,"$1,912.60","$14,948.00"
6,East,Question Marks,197,"$-38,353.69","$168,514.70"
7,East,Stars,510,"$123,012.80","$482,197.05"
8,South,Cash Cows,129,"$2,676.41","$7,025.23"
9,South,Dogs,392,"$1,066.71","$9,095.07"


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['product_count'].plot(kind='hist', bins=20, title='product_count')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('region').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_13['portfolio_quadrant_region'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_13, x='index', y='portfolio_quadrant_region', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_14['region'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_14, x='product_count', y='region', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_15['portfolio_quadrant_region'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_15, x='product_count', y='portfolio_quadrant_region', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
# prompt: Using dataframe quadrant_region_summary: create a bar chart. NOT stacked. cluster by region. color by category. height by profit.

import altair as alt
# Create a bar chart.
# Cluster bars by 'region'.
# Color the bars by 'portfolio_quadrant_region'.
# Map 'total_profit_contribution' to the height of the bars.
chart = alt.Chart(quadrant_region_summary).mark_bar().encode(
    x=alt.X('portfolio_quadrant_region', axis=None), # Map category to the x-axis, hide axis labels
    y=alt.Y('total_profit_contribution'), # Map profit to the y-axis
    color='portfolio_quadrant_region', # Color the bars by category
    column='region' # Create a separate column for each region
).properties(
    title='Total Profit Contribution by Portfolio Quadrant and Region' # Add a title to the chart
)
chart

alt.Chart(...)

In [ ]:
# prompt: Using dataframe: quadrant_region_summary How are Office Supplies, Technology, and Furniture distributed across the four quadrants? Analyze by region and by total.

import pandas as pd
import altair as alt

# Simplified SQL query for regional portfolio analysis
regional_portfolio_query = f"""
WITH ProductMetrics AS (
    SELECT
        region,
        category,
        product_name,
        SUM(profit) AS total_profit,
        SUM(sales) AS total_sales
    FROM `{FULL_TABLE_ID}`
    WHERE category IN ('Office Supplies', 'Technology', 'Furniture')
    GROUP BY region, category, product_name
),
Quartiles AS (
    SELECT *,
        NTILE(4) OVER (PARTITION BY region ORDER BY total_profit) AS profit_quartile,
        NTILE(4) OVER (PARTITION BY region ORDER BY total_sales) AS sales_quartile
    FROM ProductMetrics
)
SELECT
    region,
    category,
    product_name,
    total_profit,
    total_sales,
    CASE
        WHEN profit_quartile >= 3 AND sales_quartile >= 3 THEN 'Stars'
        WHEN profit_quartile >= 3 AND sales_quartile < 3 THEN 'Cash Cows'
        WHEN profit_quartile < 3 AND sales_quartile >= 3 THEN 'Question Marks'
        ELSE 'Dogs'
    END AS quadrant
FROM Quartiles
"""

# Execute query and get results
results_df = client.query(regional_portfolio_query).to_dataframe()

# Function to format monetary values
def format_money(df, columns):
    """Format specified columns as currency"""
    formatted_df = df.copy()
    for col in columns:
        if col in formatted_df.columns:
            formatted_df[col] = formatted_df[col].apply(
                lambda x: f"${x:,.2f}" if pd.notnull(x) else None
            )
    return formatted_df

# Analysis 1: Summary by Region, Quadrant, and Category
regional_summary = results_df.groupby(['region', 'quadrant', 'category']).agg({
    'product_name': 'count',
    'total_profit': 'sum',
    'total_sales': 'sum'
}).rename(columns={'product_name': 'product_count'}).reset_index()

# Analysis 2: Overall Summary by Quadrant and Category (across all regions)
overall_summary = results_df.groupby(['quadrant', 'category']).agg({
    'product_name': 'count',
    'total_profit': 'sum',
    'total_sales': 'sum'
}).rename(columns={'product_name': 'product_count'}).reset_index()

# Display raw results
print("=== Regional Summary (Raw Numbers) ===")
print(regional_summary)

print("\n=== Overall Summary (Raw Numbers) ===")
print(overall_summary)

# Display formatted results
print("\n=== Regional Summary (Formatted) ===")
print(format_money(regional_summary, ['total_profit', 'total_sales']))

print("\n=== Overall Summary (Formatted) ===")
print(format_money(overall_summary, ['total_profit', 'total_sales']))

# Visualization 1: Product count by quadrant and category
chart1 = alt.Chart(overall_summary).mark_bar().encode(
    x=alt.X('quadrant:O', title='Portfolio Quadrant'),
    y=alt.Y('product_count:Q', title='Number of Products'),
    color=alt.Color('category:N', title='Category'),
    tooltip=['quadrant', 'category', 'product_count', 'total_profit', 'total_sales']
).properties(
    title='Product Distribution by Quadrant and Category',
    width=400,
    height=300
)

# Visualization 2: Regional profit distribution
chart2 = alt.Chart(regional_summary).mark_bar().encode(
    x=alt.X('quadrant:O', title='Quadrant'),
    y=alt.Y('total_profit:Q', title='Total Profit'),
    color=alt.Color('category:N', title='Category'),
    column=alt.Column('region:N', title='Region'),
    tooltip=['region', 'quadrant', 'category', 'product_count', 'total_profit']
).properties(
    title='Profit by Quadrant, Category, and Region',
    width=150,
    height=200
)

# Display charts
chart1.show()
chart2.show()

=== Regional Summary (Raw Numbers) ===
     region        quadrant         category  product_count  total_profit  \
0   Central       Cash Cows        Furniture             11      206.8059   
1   Central       Cash Cows  Office Supplies            183     3004.6655   
2   Central       Cash Cows       Technology             20      282.3246   
3   Central            Dogs        Furniture             69    -1125.4759   
4   Central            Dogs  Office Supplies            322    -3007.7281   
5   Central            Dogs       Technology             43       41.9123   
6   Central  Question Marks        Furniture            113   -15234.6263   
7   Central  Question Marks  Office Supplies             74   -16207.1674   
8   Central  Question Marks       Technology             27    -2439.4860   
9   Central           Stars        Furniture             77    13282.2469   
10  Central           Stars  Office Supplies            185    25090.2099   
11  Central           Stars       Tec

alt.Chart(...)

alt.Chart(...)

In [ ]:
# prompt: Create a summary of subcategories, broken down by region. Do not create a matrix classification for this query.

import pandas as pd
# Create a summary of subcategories, broken down by region.
subcategory_region_query = f"""
SELECT
    region,
    category,
    subcategory,
    SUM(sales) AS total_sales,
    SUM(profit) AS total_profit,
    COUNT(DISTINCT product_name) AS unique_products,
    COUNT(DISTINCT order_id) AS number_of_orders
FROM
    `{FULL_TABLE_ID}`
GROUP BY
    region,
    category,
    subcategory
ORDER BY
    region,
    category,
    total_profit DESC -- Order by profit within each region and category
"""

# Run the subcategory by region query
subcategory_region_job = client.query(subcategory_region_query)

# Convert results to DataFrame
subcategory_region_df = subcategory_region_job.to_dataframe()

# Display the results
print("\n--- Subcategory Summary by Region ---")

# Optional: Format monetary columns for better readability
monetary_cols_sub_region = ['total_sales', 'total_profit']
for col in subcategory_region_df.columns:
    if col in monetary_cols_sub_region:
        subcategory_region_df[col] = subcategory_region_df[col].apply(
            lambda x: f"-${abs(x):,.2f}" if pd.notnull(x) and x < 0
            else f"${x:,.2f}" if pd.notnull(x) else None
        )

subcategory_region_df



--- Subcategory Summary by Region ---


,region,category,subcategory,total_sales,total_profit,unique_products,number_of_orders
0,Central,Furniture,Chairs,"$85,230.65","$6,592.72",70,144
1,Central,Furniture,Bookcases,"$24,157.18","-$1,997.90",32,49
2,Central,Furniture,Tables,"$39,154.97","-$3,559.65",46,68
3,Central,Furniture,Furnishings,"$15,254.37","-$3,906.22",122,191
4,Central,Office Supplies,Paper,"$17,491.90","$6,971.90",188,282
...,...,...,...,...,...,...,...
63,West,Office Supplies,Fasteners,$923.22,$275.19,31,72
64,West,Technology,Copiers,"$49,749.24","$19,327.24",13,25
65,West,Technology,Accessories,"$61,114.12","$16,484.60",118,236
66,West,Technology,Phones,"$98,684.35","$9,110.74",147,256


In [ ]:
# prompt: Using dataframe subcategory_region_df: create a stacked bar chart to visualize the top 3 subcategories in each region. place all bars on a single axis, labeled by region. bar height should indicate total profit for each subcategory. label the Y axis with currency as well.

import altair as alt

# Filter for the top 3 subcategories in each region based on the 'rank' column
top_subcategories = subcategory_region_df[subcategory_region_df['rank'] <= 5]

# Create the stacked bar chart
chart = alt.Chart(top_subcategories).mark_bar().encode(
    x=alt.X('region:N', title='Region'), # Set x-axis to region
    y=alt.Y('total_profit:Q', title='Total Profit', axis=alt.Axis(format='$,.0f')), # Set y-axis to total_profit and format as currency
    color='subcategory:N' # Stack the bars by subcategory
).properties(
    title='Top 5 Subcategories by Profit per Region' # Add a title to the chart
).properties(
    width=400,  # Increase width to 400 pixels
    height=400  # Optional: also set height
)

# Display the chart
chart

alt.Chart(...)